In [3]:
import pandas as pd
#import stockstats as StockDataFrame
import seaborn as sns
import numpy as np
import scipy.stats as stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm

from ipywidgets import *
from IPython.display import display

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
plt.rc("figure", figsize=(9, 7))
sns.set_style('whitegrid')

%matplotlib inline
pd.set_option('display.max_columns', 500)

/home/matthew/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Source: https://www.quandl.com/product/WIKIP/WIKI/PRICES-Quandl-End-Of-Day-Stocks-Info

In [4]:
# coding=utf-8
# Copyright (c) 2016, Cedric Zhuang
# All rights reserved.
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
#     * Redistributions of source code must retain the above copyright
#       notice, this list of conditions and the following disclaimer.
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#     * Neither the name of disclaimer nor the names of its contributors may
#       be used to endorse or promote products derived from this software
#       without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE REGENTS AND CONTRIBUTORS "AS IS" AND ANY
# EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
# WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE REGENTS AND CONTRIBUTORS BE LIABLE FOR ANY
# DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
# (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
# LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
# ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
# (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
# SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

from __future__ import unicode_literals

import itertools
import logging
import operator
import random
import re

import numpy as np
import pandas as pd
from int_date import get_date_from_diff

__author__ = 'Cedric Zhuang'

log = logging.getLogger(__name__)


class StockDataFrame(pd.DataFrame):
    OPERATORS = ['le', 'ge', 'lt', 'gt', 'eq', 'ne']

    KDJ_PARAM = (2.0 / 3.0, 1.0 / 3.0)

    BOLL_PERIOD = 20
    BOLL_STD_TIMES = 2

    @staticmethod
    def _get_change(df):
        df['change'] = df['close'].pct_change() * 100
        return df['change']

    @staticmethod
    def _get_p(df, column, shifts):
        """ get the permutation of specified range

        example:
        index    x   x_-2,-1_p
        0        1         NaN
        1       -1         NaN
        2        3           2  (0.x > 0, and assigned to weight 2)
        3        5           1  (2.x > 0, and assigned to weight 1)
        4        1           3

        :param df: data frame
        :param column: the column to calculate p from
        :param shifts: the range to consider
        :return:
        """
        column_name = '{}_{}_p'.format(column, shifts)
        # initialize the column if not
        df.get(column)
        shifts = StockDataFrame.to_ints(shifts)[::-1]
        indices = None
        count = 0
        for shift in shifts:
            shifted = df.shift(-shift)
            index = (shifted[column] > 0) * (2 ** count)
            if indices is None:
                indices = index
            else:
                indices += index
            count += 1
        StockDataFrame.set_nan(indices, shifts)
        df[column_name] = indices

    @classmethod
    def to_ints(cls, shifts):
        items = map(cls._process_shifts_segment,
                    shifts.split(','))
        return sorted(list(set(itertools.chain(*items))))

    @classmethod
    def to_int(cls, shifts):
        numbers = cls.to_ints(shifts)
        if len(numbers) != 1:
            raise IndexError("only accept 1 number.")
        return numbers[0]

    @staticmethod
    def to_floats(shifts):
        floats = map(float, shifts.split(','))
        return sorted(list(set(floats)))

    @classmethod
    def to_float(cls, shifts):
        floats = cls.to_floats(shifts)
        if len(floats) != 1:
            raise IndexError('only accept 1 float.')
        return floats[0]

    @staticmethod
    def _process_shifts_segment(shift_segment):
        if '~' in shift_segment:
            start, end = shift_segment.split('~')
            shifts = range(int(start), int(end) + 1)
        else:
            shifts = [int(shift_segment)]
        return shifts

    @staticmethod
    def set_nan(pd_obj, shift):
        try:
            iter(shift)
            max_shift = max(shift)
            min_shift = min(shift)
            StockDataFrame._set_nan_of_single_shift(pd_obj, max_shift)
            StockDataFrame._set_nan_of_single_shift(pd_obj, min_shift)
        except TypeError:
            # shift is not iterable
            StockDataFrame._set_nan_of_single_shift(pd_obj, shift)

    @staticmethod
    def _set_nan_of_single_shift(pd_obj, shift):
        val = np.nan
        if shift > 0:
            pd_obj.iloc[-shift:] = val
        elif shift < 0:
            pd_obj.iloc[:-shift] = val

    @classmethod
    def _get_r(cls, df, column, shifts):
        """ Get rate of change of column

        :param df: DataFrame object
        :param column: column name of the rate to calculate
        :param shifts: days to shift, accept one shift only
        :return: None
        """
        shift = cls.to_int(shifts)
        rate_key = '{}_{}_r'.format(column, shift)
        df[rate_key] = df[column].pct_change(periods=-shift) * 100

    @classmethod
    def _get_s(cls, df, column, shifts):
        """ Get the column shifted by days

        :param df: DataFrame object
        :param column: name of the column to shift
        :param shifts: days to shift, accept one shift only
        :return: None
        """
        shift = cls.to_int(shifts)
        shifted_key = "{}_{}_s".format(column, shift)
        df[shifted_key] = df[column].shift(-shift)
        StockDataFrame.set_nan(df[shifted_key], shift)

    @classmethod
    def _get_log_ret(cls, df):
        df['log-ret'] = np.log(df['close'] / df['close_-1_s'])

    @classmethod
    def _get_c(cls, df, column, shifts):
        """ get the count of column in range (shifts)

        example: kdjj_0_le_20_c
        :param df: stock data
        :param column: column name
        :param shifts: range to count, only to previous
        :return:
        """
        column_name = '{}_{}_{}'.format(column, shifts, 'c')
        shifts = abs(cls.to_int(shifts))
        df[column_name] = df[column].rolling(
            center=False, window=shifts).apply(np.count_nonzero)

    @classmethod
    def _get_op(cls, df, column, threshold, op):
        column_name = '{}_{}_{}'.format(column, threshold, op)
        threshold = cls.to_float(threshold)
        f = getattr(operator, op)
        df[column_name] = f(df[column], threshold)

    @staticmethod
    def get_diff_convolve_array(shift):
        if shift == 0:
            ret = [1]
        else:
            ret = np.zeros(abs(shift) + 1)
            if shift < 0:
                ret[[0, -1]] = 1, -1
            else:
                ret[[0, -1]] = -1, 1
        return ret

    @classmethod
    def _init_shifted_columns(cls, column, df, shifts):
        # initialize the column if not
        df.get(column)
        shifts = cls.to_ints(shifts)
        shift_column_names = ['{}_{}_s'.format(column, shift) for shift in
                              shifts]
        [df.get(name) for name in shift_column_names]
        return shift_column_names

    @classmethod
    def _get_max(cls, df, column, shifts):
        column_name = '{}_{}_max'.format(column, shifts)
        shift_column_names = cls._init_shifted_columns(column, df, shifts)
        df[column_name] = np.max(df[shift_column_names], axis=1)

    @classmethod
    def _get_min(cls, df, column, shifts):
        column_name = '{}_{}_min'.format(column, shifts)
        shift_column_names = cls._init_shifted_columns(column, df, shifts)
        df[column_name] = np.min(df[shift_column_names], axis=1)

    @staticmethod
    def _get_rsv(df, n_days):
        """ Calculate the RSV (Raw Stochastic Value) within N days

        This value is essential for calculating KDJs
        Current day is included in N
        :param df: data
        :param n_days: N days
        :return: None
        """
        n_days = int(n_days)
        column_name = 'rsv_{}'.format(n_days)
        low_min = df['low'].rolling(
            min_periods=1, window=n_days, center=False).min()
        high_max = df['high'].rolling(
            min_periods=1, window=n_days, center=False).max()

        cv = (df['close'] - low_min) / (high_max - low_min)
        df[column_name] = cv.fillna(0).astype('float64') * 100

    @staticmethod
    def _positive_sum(data):
        data = [i if i > 0 else 0 for i in data]
        ret = data[0]
        for i in data[1:]:
            ret = (ret * (len(data) - 1) + i) / len(data)
        return ret

    @staticmethod
    def _negative_sum(data):
        data = [-i if i < 0 else 0 for i in data]
        ret = data[0]
        for i in data[1:]:
            ret = (ret * (len(data) - 1) + i) / len(data)
        return ret

    # noinspection PyUnresolvedReferences
    @classmethod
    def _get_rsi(cls, df, n_days):
        """ Calculate the RSI (Relative Strength Index) within N days

        calculated based on the formula at:
        https://en.wikipedia.org/wiki/Relative_strength_index

        :param df: data
        :param n_days: N days
        :return: None
        """
        n_days = int(n_days)
        d = df['close_-1_d']

        df['closepm'] = (d + d.abs()) / 2
        df['closenm'] = (-d + d.abs()) / 2
        closepm_smma_column = 'closepm_{}_smma'.format(n_days)
        closenm_smma_column = 'closenm_{}_smma'.format(n_days)
        p_ema = df[closepm_smma_column]
        n_ema = df[closenm_smma_column]

        rs_column_name = 'rs_{}'.format(n_days)
        rsi_column_name = 'rsi_{}'.format(n_days)
        df[rs_column_name] = rs = p_ema / n_ema
        df[rsi_column_name] = 100 - 100 / (1.0 + rs)

        del df['closepm']
        del df['closenm']
        del df[closepm_smma_column]
        del df[closenm_smma_column]

    @classmethod
    def _get_smma(cls, df, column, windows):
        """ get smoothed moving average.

        :param df: data
        :param windows: range
        :return: result series
        """
        window = cls.get_only_one_positive_int(windows)
        column_name = '{}_{}_smma'.format(column, window)
        smma = df[column].ewm(
            ignore_na=False, alpha=1.0 / window,
            min_periods=0, adjust=True).mean()
        df[column_name] = smma
        return smma

    @classmethod
    def _get_trix(cls, df, column=None, windows=None):
        if column is None and windows is None:
            column_name = 'trix'
        else:
            column_name = '{}_{}_trix'.format(column, windows)

        if column is None:
            column = 'close'
        if windows is None:
            windows = 12
        window = cls.get_only_one_positive_int(windows)

        single = '{c}_{w}_ema'.format(c=column, w=window)
        double = '{c}_{w}_ema_{w}_ema'.format(c=column, w=window)
        triple = '{c}_{w}_ema_{w}_ema_{w}_ema'.format(c=column, w=window)
        df['ema3'] = df[triple]
        prev_ema3 = df['ema3_-1_s']
        df[column_name] = (df['ema3'] - prev_ema3) * 100 / prev_ema3

        del df[single]
        del df[double]
        del df[triple]
        del df['ema3']
        del df['ema3_-1_s']

    @classmethod
    def _get_wr(cls, df, n_days):
        """ Williams Overbought/Oversold Index

        WMS=[(Hn—Ct)/(Hn—Ln)] ×100
        Ct - the close price
        Hn - N days high
        Ln - N days low

        :param df: data
        :param n_days: N days
        :return: None
        """
        n_days = int(n_days)
        ln = df['low'].rolling(min_periods=1, window=n_days,
                               center=False).min()

        hn = df['high'].rolling(min_periods=1, window=n_days,
                                center=False).max()
        column_name = 'wr_{}'.format(n_days)
        df[column_name] = (hn - df['close']) / (hn - ln) * 100

    @classmethod
    def _get_cci(cls, df, n_days=None):
        """ Commodity Channel Index

        CCI = (Typical Price  -  20-period SMA of TP) / (.015 x Mean Deviation)
        Typical Price (TP) = (High + Low + Close)/3
        TP is also implemented as 'middle'.

        :param df: data
        :param n_days: N days window
        :return: None
        """
        if n_days is None:
            n_days = 14
            column_name = 'cci'
        else:
            n_days = int(n_days)
            column_name = 'cci_{}'.format(n_days)

        tp = df['middle']
        tp_sma = df['middle_{}_sma'.format(n_days)]
        md = df['middle'].rolling(
            min_periods=1, center=False, window=n_days).apply(
            lambda x: np.fabs(x - x.mean()).mean())

        df[column_name] = (tp - tp_sma) / (.015 * md)

    @classmethod
    def _get_tr(cls, df):
        """ True Range of the trading

        tr = max[(high - low), abs(high - close_prev), abs(low - close_prev)]
        :param df: data
        :return: None
        """
        prev_close = df['close_-1_s']
        high = df['high']
        low = df['low']
        c1 = high - low
        c2 = np.abs(high - prev_close)
        c3 = np.abs(low - prev_close)
        df['tr'] = np.max((c1, c2, c3), axis=0)

    @classmethod
    def _get_atr(cls, df, window=None):
        """ Average True Range

        The average true range is an N-day smoothed moving average (SMMA) of
        the true range values.  Default to 14 days.
        https://en.wikipedia.org/wiki/Average_true_range

        :param df: data
        :return: None
        """
        if window is None:
            window = 14
            column_name = 'atr'
        else:
            window = int(window)
            column_name = 'atr_{}'.format(window)
        tr_smma_column = 'tr_{}_smma'.format(window)

        df[column_name] = df[tr_smma_column]
        del df[tr_smma_column]

    @classmethod
    def _get_dma(cls, df):
        """ Different of Moving Average

        default to 10 and 50.
        :param df: data
        :return: None
        """
        df['dma'] = df['close_10_sma'] - df['close_50_sma']

    @classmethod
    def _get_dmi(cls, df):
        """ get the default setting for DMI

        including:
        +DI: 14 days SMMA of +DM,
        -DI: 14 days SMMA of -DM,
        DX: based on +DI and -DI
        ADX: 6 days SMMA of DX
        :param df: data
        :return:
        """
        df['pdi'] = cls._get_pdi(df, 14)
        df['mdi'] = cls._get_mdi(df, 14)
        df['dx'] = cls._get_dx(df, 14)
        df['adx'] = df['dx_6_ema']
        df['adxr'] = df['adx_6_ema']

    @classmethod
    def _get_um_dm(cls, df):
        """ Up move and down move

        initialize up move and down move
        :param df: data
        """
        hd = df['high_delta']
        df['um'] = (hd + hd.abs()) / 2
        ld = -df['low_delta']
        df['dm'] = (ld + ld.abs()) / 2

    @classmethod
    def _get_pdm(cls, df, windows):
        """ +DM, positive directional moving

        If window is not 1, calculate the SMMA of +DM
        :param df: data
        :param windows: range
        :return:
        """
        window = cls.get_only_one_positive_int(windows)
        column_name = 'pdm_{}'.format(window)
        um, dm = df['um'], df['dm']
        df['pdm'] = np.where(um > dm, um, 0)
        if window > 1:
            pdm = df['pdm_{}_ema'.format(window)]
        else:
            pdm = df['pdm']
        df[column_name] = pdm

    @classmethod
    def _get_vr(cls, df, windows=None):
        if windows is None:
            window = 26
            column_name = 'vr'
        else:
            window = cls.get_only_one_positive_int(windows)
            column_name = 'vr_{}'.format(window)

        df['av'] = np.where(df['change'] > 0, df['volume'], 0)
        avs = df['av'].rolling(
            min_periods=1, window=window, center=False).sum()

        df['bv'] = np.where(df['change'] < 0, df['volume'], 0)
        bvs = df['bv'].rolling(
            min_periods=1, window=window, center=False).sum()

        df['cv'] = np.where(df['change'] == 0, df['volume'], 0)
        cvs = df['cv'].rolling(
            min_periods=1, window=window, center=False).sum()

        df[column_name] = (avs + cvs / 2) / (bvs + cvs / 2) * 100
        del df['av']
        del df['bv']
        del df['cv']

    @classmethod
    def _get_mdm(cls, df, windows):
        """ -DM, negative directional moving accumulation

        If window is not 1, return the SMA of -DM.
        :param df: data
        :param windows: range
        :return:
        """
        window = cls.get_only_one_positive_int(windows)
        column_name = 'mdm_{}'.format(window)
        um, dm = df['um'], df['dm']
        df['mdm'] = np.where(dm > um, dm, 0)
        if window > 1:
            mdm = df['mdm_{}_ema'.format(window)]
        else:
            mdm = df['mdm']
        df[column_name] = mdm

    @classmethod
    def _get_pdi(cls, df, windows):
        """ +DI, positive directional moving index

        :param df: data
        :param windows: range
        :return:
        """
        window = cls.get_only_one_positive_int(windows)
        pdm_column = 'pdm_{}'.format(window)
        tr_column = 'atr_{}'.format(window)
        pdi_column = 'pdi_{}'.format(window)
        df[pdi_column] = df[pdm_column] / df[tr_column] * 100
        return df[pdi_column]

    @classmethod
    def _get_mdi(cls, df, windows):
        window = cls.get_only_one_positive_int(windows)
        mdm_column = 'mdm_{}'.format(window)
        tr_column = 'atr_{}'.format(window)
        mdi_column = 'mdi_{}'.format(window)
        df[mdi_column] = df[mdm_column] / df[tr_column] * 100
        return df[mdi_column]

    @classmethod
    def _get_dx(cls, df, windows):
        window = cls.get_only_one_positive_int(windows)
        dx_column = 'dx_{}'.format(window)
        mdi_column = 'mdi_{}'.format(window)
        pdi_column = 'pdi_{}'.format(window)
        mdi, pdi = df[mdi_column], df[pdi_column]
        df[dx_column] = abs(pdi - mdi) / (pdi + mdi) * 100
        return df[dx_column]

    @classmethod
    def _get_kdj_default(cls, df):
        """ default KDJ, 9 days

        :param df: k line data frame
        :return: None
        """
        df['kdjk'] = df['kdjk_9']
        df['kdjd'] = df['kdjd_9']
        df['kdjj'] = df['kdjj_9']

    @classmethod
    def _get_cr(cls, df, window=26):
        ym = df['middle_-1_s']
        h = df['high']
        p1_m = df.loc[:, ['middle_-1_s', 'high']].min(axis=1)
        p2_m = df.loc[:, ['middle_-1_s', 'low']].min(axis=1)
        p1 = (h - p1_m).rolling(
            min_periods=1, window=window, center=False).sum()
        p2 = (ym - p2_m).rolling(
            min_periods=1, window=window, center=False).sum()
        df['cr'] = p1 / p2 * 100
        del df['middle_-1_s']
        df['cr-ma1'] = cls._shifted_cr_sma(df, 5)
        df['cr-ma2'] = cls._shifted_cr_sma(df, 10)
        df['cr-ma3'] = cls._shifted_cr_sma(df, 20)

    @classmethod
    def _shifted_cr_sma(cls, df, window):
        name = cls._temp_name()
        df[name] = df['cr'].rolling(min_periods=1, window=window,
                                    center=False).mean()
        to_shift = '{}_-{}_s'.format(name, int(window / 2.5 + 1))
        ret = df[to_shift]
        del df[name], df[to_shift]
        return ret

    @classmethod
    def _temp_name(cls):
        return 'sdf{}'.format(random.randint(0, 10e8))

    @classmethod
    def _get_middle(cls, df):
        df['middle'] = (df['close'] + df['high'] + df['low']) / 3.0

    @classmethod
    def _calc_kd(cls, column):
        param0, param1 = cls.KDJ_PARAM
        k = 50.0
        # noinspection PyTypeChecker
        for i in param1 * column:
            k = param0 * k + i
            yield k

    @classmethod
    def _get_kdjk(cls, df, n_days):
        """ Get the K of KDJ

        K ＝ 2/3 × (prev. K) +1/3 × (curr. RSV)
        2/3 and 1/3 are the smooth parameters.
        :param df: data
        :param n_days: calculation range
        :return: None
        """
        rsv_column = 'rsv_{}'.format(n_days)
        k_column = 'kdjk_{}'.format(n_days)
        df[k_column] = list(cls._calc_kd(df.get(rsv_column)))

    @classmethod
    def _get_kdjd(cls, df, n_days):
        """ Get the D of KDJ

        D = 2/3 × (prev. D) +1/3 × (curr. K)
        2/3 and 1/3 are the smooth parameters.
        :param df: data
        :param n_days: calculation range
        :return: None
        """
        k_column = 'kdjk_{}'.format(n_days)
        d_column = 'kdjd_{}'.format(n_days)
        df[d_column] = list(cls._calc_kd(df.get(k_column)))

    @staticmethod
    def _get_kdjj(df, n_days):
        """ Get the J of KDJ

        J = 3K-2D
        :param df: data
        :param n_days: calculation range
        :return: None
        """
        k_column = 'kdjk_{}'.format(n_days)
        d_column = 'kdjd_{}'.format(n_days)
        j_column = 'kdjj_{}'.format(n_days)
        df[j_column] = 3 * df[k_column] - 2 * df[d_column]

    @staticmethod
    def remove_random_nan(pd_obj):
        return pd_obj.where((pd.notnull(pd_obj)), None)

    @staticmethod
    def _get_d(df, column, shifts):
        shift = StockDataFrame.to_int(shifts)
        shift_column = '{}_{}_s'.format(column, shift)
        column_name = '{}_{}_d'.format(column, shift)
        df[column_name] = df[column] - df[shift_column]
        StockDataFrame.set_nan(df[column_name], shift)

    @classmethod
    def _get_sma(cls, df, column, windows):
        """ get simple moving average

        :param df: data
        :param column: column to calculate
        :param windows: collection of window of simple moving average
        :return: None
        """
        window = cls.get_only_one_positive_int(windows)
        column_name = '{}_{}_sma'.format(column, window)
        df[column_name] = df[column].rolling(min_periods=1, window=window,
                                             center=False).mean()

    @classmethod
    def _get_ema(cls, df, column, windows):
        """ get exponential moving average

        :param df: data
        :param column: column to calculate
        :param windows: collection of window of exponential moving average
        :return: None
        """
        window = cls.get_only_one_positive_int(windows)
        column_name = '{}_{}_ema'.format(column, window)
        if len(df[column]) > 0:
            df[column_name] = df[column].ewm(
                ignore_na=False, span=window,
                min_periods=0, adjust=True).mean()
        else:
            df[column_name] = []

    @classmethod
    def _get_boll(cls, df):
        """ Get Bollinger bands.

        boll_ub means the upper band of the Bollinger bands
        boll_lb means the lower band of the Bollinger bands
        boll_ub = MA + Kσ
        boll_lb = MA − Kσ
        M = BOLL_PERIOD
        K = BOLL_STD_TIMES
        :param df: data
        :return: None
        """
        moving_avg = df['close_{}_sma'.format(cls.BOLL_PERIOD)]
        moving_std = df['close_{}_mstd'.format(cls.BOLL_PERIOD)]
        df['boll'] = moving_avg
        moving_avg = list(map(np.float64, moving_avg))
        moving_std = list(map(np.float64, moving_std))
        # noinspection PyTypeChecker
        df['boll_ub'] = np.add(moving_avg,
                               np.multiply(cls.BOLL_STD_TIMES, moving_std))
        # noinspection PyTypeChecker
        df['boll_lb'] = np.subtract(moving_avg,
                                    np.multiply(cls.BOLL_STD_TIMES,
                                                moving_std))

    @staticmethod
    def _get_macd(df):
        """ Moving Average Convergence Divergence

        This function will initialize all following columns

        MACD Line (macd): (12-day EMA - 26-day EMA)
        Signal Line (macds): 9-day EMA of MACD Line
        MACD Histogram (macdh): MACD Line - Signal Line
        :param df: data
        :return: None
        """
        fast = df['close_12_ema']
        slow = df['close_26_ema']
        df['macd'] = fast - slow
        df['macds'] = df['macd_9_ema']
        df['macdh'] = 2 * (df['macd'] - df['macds'])
        del df['macd_9_ema']
        del fast
        del slow

    @classmethod
    def get_only_one_positive_int(cls, windows):
        if isinstance(windows, int):
            window = windows
        else:
            window = cls.to_int(windows)
            if window <= 0:
                raise IndexError("window must be greater than 0")
        return window

    @classmethod
    def _get_mstd(cls, df, column, windows):
        """ get moving standard deviation

        :param df: data
        :param column: column to calculate
        :param windows: collection of window of moving standard deviation
        :return: None
        """
        window = cls.get_only_one_positive_int(windows)
        column_name = '{}_{}_mstd'.format(column, window)
        df[column_name] = df[column].rolling(min_periods=1, window=window,
                                             center=False).std()

    @classmethod
    def _get_mvar(cls, df, column, windows):
        """ get moving variance

        :param df: data
        :param column: column to calculate
        :param windows: collection of window of moving variance
        :return: None
        """
        window = cls.get_only_one_positive_int(windows)
        column_name = '{}_{}_mvar'.format(column, window)
        df[column_name] = df[column].rolling(
            min_periods=1, window=window, center=False).var()

    @staticmethod
    def parse_column_name(name):
        m = re.match('(.*)_([\d\-\+~,\.]+)_(\w+)', name)
        ret = [None, None, None]
        if m is None:
            m = re.match('(.*)_([\d\-\+~,]+)', name)
            if m is not None:
                ret = m.group(1, 2)
                ret = ret + (None,)
        else:
            ret = m.group(1, 2, 3)
        return ret

    CROSS_COLUMN_MATCH_STR = '(.+)_(x|xu|xd)_(.+)'

    @classmethod
    def is_cross_columns(cls, name):
        return re.match(cls.CROSS_COLUMN_MATCH_STR, name) is not None

    @classmethod
    def parse_cross_column(cls, name):
        m = re.match(cls.CROSS_COLUMN_MATCH_STR, name)
        ret = [None, None, None]
        if m is not None:
            ret = m.group(1, 2, 3)
        return ret

    @staticmethod
    def _get_rate(df):
        """ same as percent

        :param df: data frame
        :return: None
        """
        df['rate'] = df['close'].pct_change() * 100

    @staticmethod
    def _get_delta(df, key):
        key_to_delta = key.replace('_delta', '')
        df[key] = df[key_to_delta].diff()
        return df[key]

    @staticmethod
    def _get_cross(df, key):
        left, op, right = StockDataFrame.parse_cross_column(key)
        lt_series = df[left] > df[right]
        # noinspection PyTypeChecker
        different = np.zeros_like(lt_series)
        if len(different) > 1:
            # noinspection PyTypeChecker
            different[1:] = np.diff(lt_series)
            different[0] = False
        if op == 'x':
            df[key] = different
        elif op == 'xu':
            df[key] = different & lt_series
        elif op == 'xd':
            df[key] = different & ~lt_series
        return df[key]

    @staticmethod
    def init_columns(obj, columns):
        if isinstance(columns, list):
            for column in columns:
                StockDataFrame.__init_column(obj, column)
        else:
            StockDataFrame.__init_column(obj, columns)

    @classmethod
    def __init_not_exist_column(cls, df, key):
        if key == 'change':
            cls._get_change(df)
        elif key == 'rate':
            cls._get_rate(df)
        elif key == 'middle':
            cls._get_middle(df)
        elif key in ['boll', 'boll_ub', 'boll_lb']:
            cls._get_boll(df)
        elif key in ['macd', 'macds', 'macdh']:
            cls._get_macd(df)
        elif key in ['kdjk', 'kdjd', 'kdjj']:
            cls._get_kdj_default(df)
        elif key in ['cr', 'cr-ma1', 'cr-ma2', 'cr-ma3']:
            cls._get_cr(df)
        elif key in ['cci']:
            cls._get_cci(df)
        elif key in ['tr']:
            cls._get_tr(df)
        elif key in ['atr']:
            cls._get_atr(df)
        elif key in ['um', 'dm']:
            cls._get_um_dm(df)
        elif key in ['pdi', 'mdi', 'dx', 'adx', 'adxr']:
            cls._get_dmi(df)
        elif key in ['trix']:
            cls._get_trix(df)
        elif key in ['vr']:
            cls._get_vr(df)
        elif key in ['dma']:
            cls._get_dma(df)
        elif key == 'log-ret':
            cls._get_log_ret(df)
        elif key.endswith('_delta'):
            cls._get_delta(df, key)
        elif cls.is_cross_columns(key):
            cls._get_cross(df, key)
        else:
            c, r, t = cls.parse_column_name(key)
            if t is not None:
                if t in cls.OPERATORS:
                    # support all kinds of compare operators
                    cls._get_op(df, c, r, t)
                else:
                    func_name = '_get_{}'.format(t)
                    getattr(cls, func_name)(df, c, r)
            else:
                func_name = '_get_{}'.format(c)
                getattr(cls, func_name)(df, r)

    @staticmethod
    def __init_column(df, key):
        if key not in df:
            if len(df) == 0:
                df[key] = []
            else:
                StockDataFrame.__init_not_exist_column(df, key)

    def __getitem__(self, item):
        try:
            result = self.retype(
                super(StockDataFrame, self).__getitem__(item))
        except KeyError:
            try:
                self.init_columns(self, item)
            except AttributeError:
                log.exception('{} not found.'.format(item))
            result = self.retype(
                super(StockDataFrame, self).__getitem__(item))
        return result

    def in_date_delta(self, delta_day, anchor=None):
        if anchor is None:
            anchor = self.get_today()
        other_day = get_date_from_diff(anchor, delta_day)
        if delta_day > 0:
            start, end = anchor, other_day
        else:
            start, end = other_day, anchor
        return self.retype(self.ix[start:end])

    def till(self, end_date):
        return self[self.index <= end_date]

    def start_from(self, start_date):
        return self[self.index >= start_date]

    def within(self, start_date, end_date):
        return self.start_from(start_date).till(end_date)

    def copy(self, deep=True):
        return self.retype(super(StockDataFrame, self).copy(deep))

    @staticmethod
    def retype(value, index_column=None):
        """ if the input is a `DataFrame`, convert it to this class.

        :param index_column: the column that will be used as index,
                             default to `date`
        :param value: value to convert
        :return: this extended class
        """
        if index_column is None:
            index_column = 'date'

        if isinstance(value, pd.DataFrame):
            # use all lower case for column name
            value.columns = map(lambda c: c.lower(), value.columns)

            if index_column in value.columns:
                value.set_index(index_column, inplace=True)
            value = StockDataFrame(value)
        return value


## Load and explore data

In [5]:

data = pd.read_csv('WIKI_PRICES_212b326a081eacca455e13140d7bb9db.csv')

In [6]:
data.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,29.367758,32.272262,25.817810,28.399591,44739900.0
1,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,27.714581,27.753307,25.694399,26.062292,10897100.0
2,A,1999-11-22,41.31,44.00,40.06,44.00,4705200.0,0.0,1.0,26.663343,28.399591,25.856536,28.399591,4705200.0
3,A,1999-11-23,42.50,43.63,40.25,40.25,4274400.0,0.0,1.0,27.436863,28.166360,25.984323,25.984323,4274400.0
4,A,1999-11-24,40.13,41.94,40.00,41.06,3464400.0,0.0,1.0,25.905464,27.073889,25.821544,26.505815,3464400.0


In [7]:
data.tail()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
14861029,ZUMZ,2017-05-16,16.25,16.300,15.775,15.95,427139.0,0.0,1.0,16.25,16.300,15.775,15.95,427139.0
14861030,ZUMZ,2017-05-17,15.80,16.050,15.400,15.45,457527.0,0.0,1.0,15.80,16.050,15.400,15.45,457527.0
14861031,ZUMZ,2017-05-18,15.40,15.750,15.150,15.25,434859.0,0.0,1.0,15.40,15.750,15.150,15.25,434859.0
14861032,ZUMZ,2017-05-19,15.30,15.449,14.650,14.95,613783.0,0.0,1.0,15.30,15.449,14.650,14.95,613783.0
14861033,ZUMZ,2017-05-22,14.95,15.500,14.900,15.40,441440.0,0.0,1.0,14.95,15.500,14.900,15.40,441440.0


In [8]:
data.shape

(14861034, 14)

In [9]:
data.isnull().sum()

ticker           0
date             0
open           538
high            55
low             55
close            1
volume           0
ex-dividend      0
split_ratio      1
adj_open       538
adj_high        55
adj_low         55
adj_close        1
adj_volume       0
dtype: int64

In [10]:
data = data.dropna()

In [11]:
data.isnull().sum()

ticker         0
date           0
open           0
high           0
low            0
close          0
volume         0
ex-dividend    0
split_ratio    0
adj_open       0
adj_high       0
adj_low        0
adj_close      0
adj_volume     0
dtype: int64

In [12]:
data.shape

(14860480, 14)

In [13]:
import datetime
data['date'] = pd.to_datetime(data['date'])
data.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,29.367758,32.272262,25.817810,28.399591,44739900.0
1,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,27.714581,27.753307,25.694399,26.062292,10897100.0
2,A,1999-11-22,41.31,44.00,40.06,44.00,4705200.0,0.0,1.0,26.663343,28.399591,25.856536,28.399591,4705200.0
3,A,1999-11-23,42.50,43.63,40.25,40.25,4274400.0,0.0,1.0,27.436863,28.166360,25.984323,25.984323,4274400.0
4,A,1999-11-24,40.13,41.94,40.00,41.06,3464400.0,0.0,1.0,25.905464,27.073889,25.821544,26.505815,3464400.0


In [14]:
zero = data.ix[data['open'] == 0]
zero.shape

/home/matthew/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


(0, 14)

In [15]:
lessthanone = data.ix[data['open'] <= 10]
lessthanone.shape

(3118132, 14)

In [16]:
smallvolume = data.ix[data['volume'] >= 10000000]
smallvolume.shape

(277001, 14)

In [17]:
data['marketopen'] = data['volume'] * data['open']
data['marketclose'] = data['volume'] * data['close']
data['marketgain'] = data['marketclose'] - data['marketopen']
data['volatility'] = (data['high'] - data['low']) / ((data['high'] + data['low']) / 2)

In [18]:
data.sort_values('volatility', ascending = False)

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume,marketopen,marketclose,marketgain,volatility
6906243,IG,2008-12-05,0.65000,9999.00000,0.650000,0.74000,11200.0,0.0,1.000000,0.650000,9999.000000,0.650000,0.740000,11200.0,7.280000e+03,8.288000e+03,1.008000e+03,1.999740
11317818,RHI,2006-07-14,36.57000,50.00000,0.010000,35.49000,3323000.0,0.0,1.000000,30.070311,41.113359,0.008223,29.182262,3323000.0,1.215221e+08,1.179333e+08,-3.588840e+06,1.999200
9816796,ODP,2002-12-04,0.01000,16.48000,0.010000,16.30000,5774200.0,0.0,1.000000,0.009830,16.199886,0.009830,16.022945,5774200.0,5.774200e+04,9.411946e+07,9.406172e+07,1.997574
10964438,PTX,2008-12-03,0.86000,1.00000,0.010000,0.98000,12000.0,0.0,1.000000,17.200000,20.000000,0.200000,19.600000,600.0,1.032000e+04,1.176000e+04,1.440000e+03,1.960396
14117532,VTNR,2009-05-06,1.00000,1.00000,0.015000,1.00000,1000.0,0.0,1.000000,1.000000,1.000000,0.015000,1.000000,1000.0,1.000000e+03,1.000000e+03,0.000000e+00,1.940887
10255499,PCYG,2003-02-21,0.00100,0.06000,0.001000,0.04000,20000.0,0.0,1.000000,0.050000,3.000000,0.050000,2.000000,400.0,2.000000e+01,8.000000e+02,7.800000e+02,1.934426
12696239,SWY,2000-06-23,42.00000,43.25000,0.812500,42.06000,2696900.0,0.0,1.000000,31.021998,31.945272,0.600128,31.066315,2696900.0,1.132698e+08,1.134316e+08,1.618140e+05,1.926241
14558245,WSTL,2008-09-29,0.74500,0.74500,0.020000,0.65000,472800.0,0.0,1.000000,0.745000,0.745000,0.020000,0.650000,472800.0,3.522360e+05,3.073200e+05,-4.491600e+04,1.895425
14855242,ZQK,2009-01-13,1.72000,1.84000,0.070000,1.63000,2153300.0,0.0,1.000000,1.720000,1.840000,0.070000,1.630000,2153300.0,3.703676e+06,3.509879e+06,-1.937970e+05,1.853403
13497852,TZOO,2002-10-16,1.75000,6.50000,0.250000,6.50000,300.0,0.0,1.000000,1.750000,6.500000,0.250000,6.500000,300.0,5.250000e+02,1.950000e+03,1.425000e+03,1.851852


In [19]:
data['daily_change'] = data['close'] - data['open']

In [20]:
data.sort_values('daily_change', ascending = False)

MemoryError: 

In [ ]:
data['dailyrate'] = data['close'] / data['open'] - 1

In [ ]:
data.sort_values('dailyrate', ascending = False)

In [ ]:
data['ticker'].value_counts()

In [ ]:
data.shape

In [ ]:
data = data.ix[data['open'] > 0]

In [ ]:
data.shape

In [ ]:
notpenny = data.ix[data['open'] > 1 ]

In [ ]:
notpenny.sort_values('open', ascending = True).head()

In [ ]:
notpenny['ticker'].value_counts()

In [ ]:
notpenny.shape

In [ ]:
data.shape

In [ ]:
print "number of stocks: 14860480"
print "number of penny stocks: " 
14860480 - 14649473

In [ ]:
sample1 = data.sample(n=1000000, random_state = 234)
stock = StockDataFrame.retype(sample1)

In [ ]:
stock.head()

In [ ]:
stock['macd'] = stock['macd']

In [ ]:
stock['volume_delta'] = stock['volume_delta']
stock['open_-2_r'] = stock['open_-2_r']
stock['open_2_d'] = stock['open_2_d']
stock['cr'] = stock['cr']
stock['cr-ma1'] = stock['cr-ma1']
stock['cr-ma2'] = stock['cr-ma2']
stock['cr-ma3'] = stock['cr-ma3']

In [ ]:
stock['volume_-3,2,-1_max'] = stock['volume_-3,2,-1_max']
#stock['volume_-3~1_min'] = stock['volume_-3~1_min']
stock['boll'] = stock['boll']
stock['boll_ub'] = stock['boll_ub']
stock['boll_lb'] = stock['boll_lb']
#stock['close_10.0_le_5_c'] = stock['close_10.0_le_5_c']
#stock['cr-ma2_xu_cr-ma1_20_c'] = stock['cr-ma2_xu_cr-ma1_20_c']
#stock['rsi_6'] = stock['rsi_6']

In [ ]:
stock.columns

In [ ]:
stock.head()

In [ ]:
stock['bollstatus'] = (stock['open'] - stock['boll_lb']) / (stock['boll_ub'] - stock['boll_lb'])

In [ ]:
stock.isnull().sum()

In [ ]:
stock = stock.dropna()

In [ ]:
stock.isnull().sum()

In [ ]:
stock.dtypes

In [ ]:
stock.shape

In [ ]:
c = stock.corr()
s = c.unstack()
so = s.order(kind="quicksort")
print so['dailyrate']

In [ ]:
iris = stock[['marketgain', 'volatility','bollstatus','dailyrate','daily_change']]
g = sns.pairplot(iris)

In [ ]:
14860455 * 100 / 14861034

In [ ]:
14861034 - 14860455

## Basic regression

In [ ]:
#dropped = ['ticker','open','close','high','low','adj_open','adj_high','adj_low','adj_close','daily_change','dailyrate']
#X = stock.drop(dropped, 1)
X = stock[['marketgain', 'volatility','bollstatus','daily_change']]
y = stock['dailyrate']

In [ ]:
X.columns

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import cross_val_score
linreg = LinearRegression().fit(X, y)
#logreg = LogisticRegression().fit(X, y)

In [ ]:
reg_scores = cross_val_score(LinearRegression(), X, y, cv=4)
print reg_scores, np.mean(reg_scores)

In [ ]:
# Note the difference in argument order
# optionally, you can chain "fit()" to the model object
model = sm.OLS(y, X)
model = model.fit()
predictions = model.predict(X)

# Plot the model
plt.figure(figsize=(8,6))
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values - $\hat{y}$")
plt.ylabel("Actual Values - $y$")
plt.show()

print "MSE:", mean_squared_error(y, predictions)

In [ ]:
model.summary()

# Momentum

In [ ]:
def middle_of_month(name):
    name['date'] = name.index
    stockmiddle = name.loc[:,('date','open')]
    stockmiddle['date'] = pd.DatetimeIndex(stockmiddle['date']) + pd.DateOffset(-14)
    stockmiddle['14 day price'] = stockmiddle['open']
    stockmiddle.set_index('date', drop=False, inplace=True)
    stockmiddle.drop('open', 1, inplace=True)
    stockmiddle = stockmiddle.sample(n = 10000, random_state = 10)
    stock30 = name.loc[:,('date','open')]
    stock30['date'] = pd.DatetimeIndex(stock30['date']) + pd.DateOffset(-28)
    stock30['28 day price'] = stock30['open']
    stock30.set_index('date', drop=False, inplace=True)
    stock30.drop('open', 1, inplace=True)
    stock30 = stock30.sample(n = 10000, random_state = 10)
    stock60 = name.loc[:,('date','open')]
    stock60['date'] = pd.DatetimeIndex(stock60['date']) + pd.DateOffset(-63)
    stock60['63 day price'] = stock60['open']
    stock60.set_index('date', drop=False, inplace=True)
    stock60.drop('open', 1, inplace=True)
    stock60 = stock60.sample(n = 10000, random_state = 10)
    name.set_index('date', drop=False, inplace=True)
    output = pd.merge(name,stockmiddle, left_on='date', right_on='date')
    output = pd.merge(output,stock30, left_on='date', right_on='date')
    output = pd.merge(output,stock60, left_on='date', right_on='date')
    output['2 week gain'] = (output['14 day price'] / output['open']) - 1
    output['4 week gain'] = (output['28 day price'] / output['open']) - 1
    output['7 week gain'] = (output['63 day price'] / output['open']) - 1
    output.set_index('date', drop=False, inplace=True)
    return output

In [ ]:
stock.head()

In [ ]:
middle = middle_of_month(stock)

In [ ]:
middle.head()

In [ ]:
X = middle.drop(['7 week gain','ticker','63 day price','date'], 1)
y = middle['7 week gain']
#reg_scores = cross_val_score(LinearRegression(), X, y, cv=4)
#print reg_scores, np.mean(reg_scores)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X, y)
names = X.dtypes.index
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
             reverse=True)

In [ ]:
# Note the difference in argument order
# optionally, you can chain "fit()" to the model object
model = sm.OLS(y, X)
model = model.fit()
predictions = model.predict(X)

# Plot the model
plt.figure(figsize=(8,6))
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values - $\hat{y}$")
plt.ylabel("Actual Values - $y$")
plt.show()

print "MSE:", mean_squared_error(y, predictions)

In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i] > 0 and y_hat[i] > 0:
           TP += 1
    for i in range(len(y_hat)): 
        if y_hat[i] > 0 and y_actual[i] < 0:
           FP += 1
    for i in range(len(y_hat)): 
        if y_actual[i] < 0 and  y_hat[i] < 0:
           TN += 1
    for i in range(len(y_hat)): 
        if y_hat[i] < 0 and y_actual[i] > 0:
           FN += 1
    return(TP, FP, TN, FN)

In [ ]:
success = (TP + TN) / (TP+ TN + FP + FN)

# High volume

In [ ]:
data.describe()

In [ ]:
highvolume = data.loc[data['volume'] > 6981000]

In [ ]:
highvolume.shape

In [ ]:
data.shape

In [ ]:
440325 * 100 / 14860480

In [ ]:
stock = StockDataFrame.retype(highvolume)
stock['macd'] = stock['macd']
stock['volume_delta'] = stock['volume_delta']
stock['open_-2_r'] = stock['open_-2_r']
stock['open_2_d'] = stock['open_2_d']
stock['cr'] = stock['cr']
stock['cr-ma1'] = stock['cr-ma1']
stock['cr-ma2'] = stock['cr-ma2']
stock['cr-ma3'] = stock['cr-ma3']
stock['volume_-3,2,-1_max'] = stock['volume_-3,2,-1_max']
stock['boll'] = stock['boll']
stock['boll_ub'] = stock['boll_ub']
stock['boll_lb'] = stock['boll_lb']

In [ ]:
stock.isnull().sum()

In [ ]:
stock = stock.dropna()

In [ ]:
stock.isnull().sum()

In [ ]:
dropped = ['ticker','open','close','high','low','adj_open','adj_high','adj_low','adj_close','daily_change','dailyrate']
X = stock.drop(dropped, 1)
y = stock['dailyrate']

In [ ]:
model = sm.OLS(y, X)
model = model.fit()
predictions = model.predict(X)

# Plot the model
plt.figure(figsize=(8,6))
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values - $\hat{y}$")
plt.ylabel("Actual Values - $y$")
plt.show()

print "MSE:", mean_squared_error(y, predictions)

In [ ]:
model.summary()

In [ ]:
X_large = stock[['volatility','close_26_ema','macd','macdh','open_-2_r','open_2_s','open_2_d','middle','cr','cr-ma2','close_20_sma','close_20_mstd','boll','boll_ub','boll_lb']]

In [ ]:
model = sm.OLS(y, X_large)
model = model.fit()
predictions = model.predict(X_large)

# Plot the model
plt.figure(figsize=(8,6))
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values - $\hat{y}$")
plt.ylabel("Actual Values - $y$")
plt.show()

print "MSE:", mean_squared_error(y, predictions)

In [ ]:
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.4, random_state=0)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
#clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)

In [ ]:
#clf.score(X_test, y_test)   

Set up some text data as well to predict performance.

## Small Volume

In [ ]:
smallvolume = data.loc[data['volume'] < 3670]

In [ ]:
stock = StockDataFrame.retype(smallvolume)
stock['macd'] = stock['macd']
stock['volume_delta'] = stock['volume_delta']
stock['open_-2_r'] = stock['open_-2_r']
stock['open_2_d'] = stock['open_2_d']
stock['cr'] = stock['cr']
stock['cr-ma1'] = stock['cr-ma1']
stock['cr-ma2'] = stock['cr-ma2']
stock['cr-ma3'] = stock['cr-ma3']
stock['volume_-3,2,-1_max'] = stock['volume_-3,2,-1_max']
stock['boll'] = stock['boll']
stock['boll_ub'] = stock['boll_ub']
stock['boll_lb'] = stock['boll_lb']

In [ ]:
stock.isnull().sum()

In [ ]:
stock = stock.dropna()

In [ ]:
dropped = ['ticker','open','close','high','low','adj_open','adj_high','adj_low','adj_close','daily_change','dailyrate']
X = stock.drop(dropped, 1)
y = stock['dailyrate']

In [ ]:
print "X shape: "
print X.shape
print "y shape: "
print y.shape

In [ ]:
model = sm.OLS(y, X)
model = model.fit()
predictions = model.predict(X)

# Plot the model
plt.figure(figsize=(8,6))
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values - $\hat{y}$")
plt.ylabel("Actual Values - $y$")
plt.show()

print "MSE:", mean_squared_error(y, predictions)

In [ ]:
model.summary()

In [ ]:
small_drop = ['close_12_ema', 'close_26_ema','macd','macds','macdh','cr','cr-ma1','cr-ma2','cr-ma3','close_20_sma','close_20_mstd','boll','boll_ub','boll_lb']
X.drop(small_drop, 1)

In [ ]:
model = sm.OLS(y, X)
model = model.fit()
predictions = model.predict(X)

# Plot the model
plt.figure(figsize=(8,6))
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values - $\hat{y}$")
plt.ylabel("Actual Values - $y$")
plt.show()

print "MSE:", mean_squared_error(y, predictions)

In [ ]:
model.summary()

# Daily Spread

## Large

In [ ]:
stock = StockDataFrame.retype(highvolume)
stock['macd'] = stock['macd']
stock['volume_delta'] = stock['volume_delta']
stock['open_-2_r'] = stock['open_-2_r']
stock['open_2_d'] = stock['open_2_d']
stock['cr'] = stock['cr']
stock['cr-ma1'] = stock['cr-ma1']
stock['cr-ma2'] = stock['cr-ma2']
stock['cr-ma3'] = stock['cr-ma3']
stock['volume_-3,2,-1_max'] = stock['volume_-3,2,-1_max']
stock['boll'] = stock['boll']
stock['boll_ub'] = stock['boll_ub']
stock['boll_lb'] = stock['boll_lb']

In [ ]:
stock = stock.dropna()
stock.isnull().sum()

In [ ]:
dropped = ['ticker','open','close','high','low','adj_open','adj_high','adj_low','adj_close','daily_change','dailyrate','volatility']
X = stock.drop(dropped, 1)
y = stock['volatility']

In [ ]:
model = sm.OLS(y, X)
model = model.fit()
predictions = model.predict(X)

# Plot the model
plt.figure(figsize=(8,6))
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values - $\hat{y}$")
plt.ylabel("Actual Values - $y$")
plt.show()

print "MSE:", mean_squared_error(y, predictions)

In [ ]:
model.summary()

## Restaurant Data

In [ ]:
name = pd.read_csv('2017q1/sub.txt',sep='\t')

In [ ]:
name.head()

In [ ]:
dataframe = name.merge(numbers, left_on = 'adsh', right_on = 'adsh', how='outer')

In [ ]:
dataframe.head()